In [1]:
# Imports
import numpy as np
# import altair as alt
# import pydot
from IPython.display import display, SVG, clear_output, HTML
# import matplotlib.pyplot as plt
from pydrake.all import *
from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import(
    StartMeshcat, MeshcatJointSlidersThatPublish)
from manipulation.scenarios import AddMultibodyTriad

# Import pygmo for optimisation
import pygmo as pg
import pandas as pd
import time
from TwoLink_urdf_creator import *
# from numba import jit
from joblib import Parallel, delayed
from utils import *
import dill as pickle
import time

In [2]:
# meshcat = StartMeshcat()

In [3]:
%%time

# Define the number of actuators
na = 2

# GLOBAL PROBLEM SETUP

# Reference: https://wiki.ros.org/pr2_controller_manager/safety_limits
# Defining the actuator box
class ActuatorBox(LeafSystem):
    # Input values are for motor and NOT for the joint
    def __init__(self, tau_max_m, omega_max_m, gear_ratio, k_velocity):
        LeafSystem.__init__(self)
        # Both the values are motor specific and NOT joint specific
        # Get joint max torque
        self.tau_max_j = tau_max_m*gear_ratio
        self.k_vel = k_velocity
        # Convert RPM of motor to joint velocity in rad/sec
        self.omega_max_j = omega_max_m*(2*np.pi/60)/gear_ratio
        self.DeclareVectorInputPort('commanded_torque', BasicVector(na))
        self.DeclareVectorInputPort('joint_state', BasicVector(2*na))
        self.DeclareVectorOutputPort('realised_torque', BasicVector(na), self.OutputJointTorque)

    def OutputJointTorque(self, context, output):
        # Get the state values
        q = self.EvalVectorInput(context, 1).get_value()
        qvel = q[na:]
        u = self.EvalVectorInput(context, 0).get_value()
        # Compute the instantaneous torque limit
        # For the AK-70-10 motors
        kv = self.k_vel
        u_new = np.copy(u)
        # Using URDF safety controller rules
        for i in range(len(u)):
            if qvel[i]>=0 and u[i]>=0:
                u_bound = -kv*(qvel[i]-self.omega_max_j)
                u_new[i] = np.min([u[i], u_bound, self.tau_max_j])
            if qvel[i]>=0 and u[i]<=0:
                u_bound = -self.tau_max_j
                u_new[i] = np.max([u[i], u_bound])
            if qvel[i]<=0 and u[i]>=0:
                u_bound = self.tau_max_j
                u_new[i] = np.min([u[i], u_bound])
            if qvel[i]<=0 and u[i]<=0:
                u_bound = -(-kv*(np.abs(qvel[i])-self.omega_max_j))
                u_new[i] = np.max([u[i], u_bound, -self.tau_max_j])
#             print(u[i], u_bound, u_new[i])
        output.SetFromVector(u_new)

# Task definition
# Home position
carti = np.array([-0.22, 0.14, 1.00])
qi = np.array([0.93, 0.45])
cartd = np.array([0.18, 0.15, 1.00])
qd = np.array([0.01, 0.22])
qint = qd

# Simulation settings
time_step = 1e-4
sim_time = 5
error_threshold = 5e-3

# Robot URDF
urdf_path = './urdfs/TwoLinkRobot.urdf'
print(urdf_path)

# motor mass
m_m = 0.452
# Assumed values
tau_mu = 5.3e-3
b0 = 1.5e-5
# AK70-10 parameters
# gear ratio
gear_ratio = 10
# motor design variables
tau_max_m = 12/gear_ratio # Nm
omega_max_m = 475*gear_ratio   # RPM
Kt = 0.095        # Nm/A
Kv = 100          # RPM/V
Km = 0.19         # Nm/sqrt(W)
# Computed matrix for joule loss
K_joule = 1/np.sqrt(Km)*np.identity(na)
k_velocity = 8.33/(75*2*np.pi/60) # Nm/(rad/s)
# Control attributes
Kp = 15
Kd = 2*np.sqrt(Kp)

study = ["baseline_control", "baseline_motor", "baseline_codesign", "baseline_codesign_heuristics", "baseline"]

case = study[0]
if case=="baseline_control":
    # For baseline-control study
    pre_val_vals = [tau_mu, b0, Km, k_velocity, tau_max_m, omega_max_m, gear_ratio, m_m]
    pre_val_names = ["tau_mu", "b0", "Km", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio", "m_m"]
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = ["Kp", "Kd"]
    print(case)
    print('pre_val: ', pre_val, 'dv_names: ', dv_names)
    sample_dv = [15, 2*np.sqrt(15)]
    print(sample_dv)
    dv_bounds = ([0,0],[100,100])
    print('dv_bounds: ', dv_bounds)
    
if case=="baseline_motor":
    # For baseline-control-motor study
    pre_val_vals = []
    pre_val_names = []
    print(case)
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = ["tau_mu", "b0", "Km", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio", "Kp", "Kd", "m_m"]
    print('pre_val: ', pre_val, 'dv_names: ', dv_names)
    sample_dv = [0.0053, 1.5e-05, 0.19, 1.0606085407643906, 1.2, 4750, 10, 15, 2*np.sqrt(15), 0.452]
    print('Sample-DV: ', sample_dv)
    # Since we want to change the motor mass in the simulation here, we must include joint mass variation in the URDF
    # ['tau_mu', 'b0', 'Km', 'k_velocity', 'tau_max_m', 'omega_max_m', 'gear_ratio', 'Kp', 'Kd', 'm_m']
    dv_bounds = ([0,0,0,1e-2,1e-3,1000,0.1, 0, 0, 0],[1,1,5,1e2,5,9000,25,100,100,5])
    print('dv_bounds: ', dv_bounds)
    
if case=="baseline_codesign":
    # For baseline-codesign study
    pre_val_vals = []
    pre_val_names = []
    print(case)
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = ["tau_mu", "b0", "Km", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio", "Kp", "Kd", "m_m", "l1", "l2"]
    print('pre_val: ', pre_val, 'dv_names: ', dv_names)
    sample_dv = [0.0053, 1.5e-05, 0.19, 1.0606085407643906, 1.2, 4750, 10, 15, 2*np.sqrt(15), 0.452, 0.36, 0.36]
    print('Sample-DV: ', sample_dv)
    # Since we want to change the motor mass in the simulation here, we must include joint mass variation in the URDF
    # ['tau_mu', 'b0', 'Km', 'k_velocity', 'tau_max_m', 'omega_max_m', 'gear_ratio', 'Kp', 'Kd', 'm_m']
    dv_bounds = ([0,0,0,1e-2,1e-3,1000,0.1, 0, 0, 0,1e-2,1e-2],[1,1,5,1e2,5,9000,25,100,100,5,2,2])
    print('dv_bounds: ', dv_bounds)

if case=="baseline_codesign_heuristics":
    # For baseline-codesign study
    pre_val_names = ["tau_mu", "b0", "k_velocity"]
    pre_val_vals = [0.0053, 1.5e-05, 1.0606085407643906]
    print(case)
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = ["omega_max_m", "gear_ratio", "Kp", "Kd", "m_m", "l1", "l2"]
    print('pre_val: ', pre_val, 'dv_names: ', dv_names)
    sample_dv = [4750, 10, 15, 2*np.sqrt(15), 0.452, 0.36, 0.36]
    print('Sample-DV: ', sample_dv)
    # Since we want to change the motor mass in the simulation here, we must include joint mass variation in the URDF
    # ['tau_mu', 'b0', 'Km', 'k_velocity', 'tau_max_m', 'omega_max_m', 'gear_ratio', 'Kp', 'Kd', 'm_m']
    dv_bounds = ([1000,0.1, 0, 0, 0,1e-2,1e-2],[9000,25,100,100,5,2,2])
    print('dv_bounds: ', dv_bounds)  
    
if case=="baseline":
    print(case)
    # For simulation study
    pre_val_vals = [tau_mu, b0, Km, k_velocity, tau_max_m, omega_max_m, gear_ratio, Kp, Kd, m_m]
    pre_val_names = ["tau_mu", "b0", "Km", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio", "Kp", "Kd", "m_m"]
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = []
    sample_dv = []
    print(pre_val)

./urdfs/TwoLinkRobot.urdf
baseline_control
pre_val:  {'tau_mu': 0.0053, 'b0': 1.5e-05, 'Km': 0.19, 'k_velocity': 1.0606085407643906, 'tau_max_m': 1.2, 'omega_max_m': 4750, 'gear_ratio': 10, 'm_m': 0.452} dv_names:  ['Kp', 'Kd']
[15, 7.745966692414834]
dv_bounds:  ([0, 0], [100, 100])
CPU times: user 0 ns, sys: 1.34 ms, total: 1.34 ms
Wall time: 886 µs


In [4]:
def compute_QoI(pre_val, dv_names, sample_dv, case, run_sim=False):
    # Construct the master dv list
    dv_dict = dict(zip(dv_names, sample_dv))
    if case==study[3]:
        # Map the entire design space from the sampled DVs using heuristics
        # Current heuristics used are from the jump paper
        dv_dict['tau_max_m'] = 5.48*dv_dict['m_m']**0.97
        dv_dict['Km'] = 0.15*dv_dict['m_m']**1.39
        # Later also fit k_velocity values
    dv = {**pre_val, **dv_dict}
    # Construct a new URDF if its such a case
#     print(case)
    if (case==study[1]) or (case==study[2]) or (case==study[3]):
        urdf_path=urdf_creator(dv)
    else:
        urdf_path = './urdfs/TwoLinkRobot.urdf'
    # Construct the sim diagram
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    X_R = RigidTransform(RotationMatrix.MakeYRotation(0), np.array([-0.1, 0.5, 1]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    if run_sim:
        print(urdf_path)
        meshcat.Delete()
        meshcat.DeleteAddedControls()
        # Spawn table
        table = Parser(plant, scene_graph).AddModelFromFile('./urdfs/table/extra_heavy_duty_table_modified.sdf','table')

    # Due to the way the problem is formulated, spheres and the table can be safely ignored
    plant.Finalize()
    actuator = builder.AddSystem(ActuatorBox(dv['tau_max_m'], dv['omega_max_m'], dv['gear_ratio'], dv['k_velocity']))
    
    if run_sim:
        visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    
    # Initialise controller params, for now limited to Kp and Kd search
    Kp = np.full(na, dv['Kp'])
    Ki = np.full(na, 0)
    Kd = np.full(na, dv['Kd'])
    
    iiwa_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
    iiwa_controller.set_name("iiwa_controller");
    # Complete connections
    builder.Connect(plant.get_state_output_port(model),
                iiwa_controller.get_input_port_estimated_state())
    builder.Connect(iiwa_controller.get_output_port_control(), 
                actuator.get_input_port(0))
    builder.Connect(plant.get_state_output_port(model),
                actuator.get_input_port(1))
    builder.Connect(actuator.get_output_port(), plant.get_actuation_input_port())
    
    # Connecting a data logger
    # Commanded torque
    logger1 = LogVectorOutput(iiwa_controller.get_output_port_control(), builder)
    # Joint state
    logger2 = LogVectorOutput(plant.get_state_output_port(model), builder)
    # Realised torque
    logger3 = LogVectorOutput(actuator.get_output_port(), builder)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    gripper_frame = plant.GetBodyByName("eef").body_frame()
    
    # Now computing the QoIs
    # WORKSPACE
    # Workspace is always satisfied since its the same robot
    workspace = 0
    # DEFLECTION
    # Deflection is not applicable as we consider only rigid components
    deflection = 0
    # APPLIED FORCE
    # Applied force will also remain the same given that the tau_max is fixed
    applied_force = 0
    # PAYLOAD
    # same as applied force
    payload = 0
    # Setup the simulation
    xd = np.hstack((qint, 0*qint))
    plant.SetPositions(plant_context, qi)
    iiwa_controller.GetInputPort('desired_state').FixValue(
    iiwa_controller.GetMyMutableContextFromRoot(context), xd);
    # Simulation
    simulator = Simulator(diagram, context)
    simulator.AdvanceTo(sim_time);
    
    # Now we want to get the simulation data, i.e., 
    # the torque applied, final time of the joint variables
    log1 = logger1.FindLog(context)
    log2 = logger2.FindLog(context)
    log3 = logger3.FindLog(context)
    
    error = np.abs(log2.data()[:na,:].transpose() - np.tile(qint,(log3.data().shape[1],1)))
    e_norm = np.array([np.linalg.norm(val) for val in error])
    for i in range(len(e_norm)):
        if np.all(e_norm[i:]<error_threshold):
            break
    final_error = e_norm[-1]
    # CYCLE TIME
    ts = log2.sample_times()[i]
    
    omega_j = []
    for i in range(log2.data().shape[1]):
        omega_j.append(log2.data()[:,i][na:])
    omega_j = np.array(omega_j).transpose()
    omega_m = omega_j*dv['gear_ratio']
    # Convert the motor speed to RPM
    omega_m_rpm = omega_m*60/(2*np.pi)
    # Motor torque realised
    tau_m = log3.data()/dv['gear_ratio']
    
    K_joule = 1/np.sqrt(dv['Km'])*np.identity(na)
    # Total power losses
    inst_friction_power = []
    inst_joule_power = []
    for i in range(len(log3.sample_times())):
        omega_inst = omega_m[:, i]
        tau_f = dv['tau_mu']*np.sign(omega_inst)+dv['b0']*omega_inst
        P_f = np.dot(tau_f, omega_inst)
        inst_friction_power.append(P_f)
        tau_t = tau_f+tau_m[:, i]
        P_t = np.dot(K_joule.dot(tau_t), tau_t)
        inst_joule_power.append(P_t)
    
    # POWER LOSSES
    total_friction_loss = np.sum(inst_friction_power)*time_step
    total_joule_loss = np.sum(inst_joule_power)*time_step
    
    results = np.array([workspace, deflection, applied_force, payload, ts, total_friction_loss, total_joule_loss, final_error])
#     print(results)
    
    if run_sim:
        # Angular angular velocity of the motors in RPM
        motor_rpm = omega_j*gear_ratio*60/(2*np.pi)
        plt.plot(log2.sample_times(), motor_rpm[0, :], log2.sample_times(), motor_rpm[1, :])
        plt.title('motor RPM vs time')

        # Motor torque demanded
        fig = plt.figure()
        plt.plot(log1.sample_times(), log1.data()[0, :], log1.sample_times(), log1.data()[1, :])
        plt.title("motor torque demanded vs time")

        # Motor torque demanded
        fig = plt.figure()
        plt.plot(log1.sample_times(), log3.data()[0, :], log1.sample_times(), log3.data()[1, :])
        plt.title("motor torque realised vs time")

        print('Max realised torque', np.max(np.abs(log3.data())))

    return results

In [5]:
# test with a sample or for sim
# print(sample_dv)
# compute_QoI(pre_val, dv_names, sample_dv, case)

In [6]:
# Setting up a problem
class Baseline_Problem:
    def __init__(self, pre_val, dv_names, dv_bounds, case):
        self.pre_val = pre_val
        self.dv_names = dv_names
        self.dv_bounds = dv_bounds
        self.case = case
        pass

    def dim(self):
        if self.case==study[0]:
            return 2
        else:
            return 0
        pass
    
    
    def fitness(self,sample_dv):
#         print(sample_dv)
        # Write a function to output the QoI values
        results = compute_QoI(self.pre_val, self.dv_names, sample_dv, self.case)
        # Add weighting factor, K_w
        # workspace, deflection, applied_force, payload, ts, total_friction_loss, total_joule_loss, final_error
        K_w = np.diag([1,1,1,1,100,5,5,0])
        # Quadratic objective meta function with weights
        fitness_val = np.matmul(K_w, results).dot(results)
        return [fitness_val]

    def get_bounds(self):
        # Define bounds on the controller variables
        # Ideally from a csv file like before so that one can easily extend the optimisation problem
        return self.dv_bounds

    def get_name(self):
        # get the name of the problem also from the csv sheet
        return "Baseline_Control"

    def batch_fitness(self, dvs):
        # print(dvs)
#         First, reshape the values into an array
        new_dvs = np.reshape(dvs, (2, 70)).T
        var_list = new_dvs.tolist()
        # print(var_list)
#         Now use the joblib style to evaluate the fitness
        with tqdm_joblib(tqdm(total=len(var_list))) as progress_bar:
            results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(self.fitness)(var_list[i]) for i in range(len(var_list)))
        results = list(itertools.chain(*results))
        # print(results)
        return results

#         Define custom batch fitness evaluation using joblib for efficient parallelisation
        pass

    def has_batch_fitness(self):
#         Info if the batch fitness is available
        return True

In [7]:
bla = Baseline_Problem(pre_val, dv_names, dv_bounds, case)

In [8]:
prob = pg.problem(Baseline_Problem(pre_val, dv_names, dv_bounds, case))
# print(prob)
# print(prob.get_bounds())

In [9]:
# # Using CMA-ES
algo = pg.algorithm(pg.cmaes(gen = 10, sigma0=0.3))
algo.set_verbosity(100)
# print(algo)

In [10]:
# Writing a custom island to see if we can make it work
class cool_isl:
    def run_evolve(self, algo, pop):
        # Evolve is not in our control to modify
        with futures.ThreadPoolExecutor(max_workers=12) as executor:
            future = executor.submit(algo.evolve, pop)
#             new_pop = algo.evolve(pop)
#         return algo, new_pop
        return algo, future.result()
    def get_name(self):
        return "This is a cool island that works"  

In [11]:
%%time
archi = pg.archipelago(n=1, algo = algo, prob=prob, pop_size=70, b = pg.default_bfe(), udi = cool_isl())
# archi = pg.archipelago(n=1, algo = algo, prob=prob, pop_size=5)
archi.evolve(); archi.wait_check()

100%|███████████████████████████████████████████| 70/70 [00:48<00:00,  1.44it/s]


CMAES 4 PaGMO: 
mu: 35 - lambda: 70 - mueff: 19.187 - N: 2
cc: 0.539703 - cs: 0.809066 - c1: 0.0664959 - cmu: 0.979859 - sigma: 0.3 - damps: 4.73343 - chiN: 1.25427

   Gen:        Fevals:          Best:            dx:            df:         sigma:
      1              0        43.0439        23.9516        2461.97            0.3
Exit condition -- generations = 10
CPU times: user 1h 4min 32s, sys: 1min 55s, total: 1h 6min 27s
Wall time: 1h 3min 37s


In [12]:
a = archi.get_champions_f()
a2 = sorted(archi.get_champions_f(), key = lambda x: x[0])[0]
best_isl_idx = [(el == a2).all() for el in a].index(True)
x_best = archi.get_champions_x()[best_isl_idx]
f_best = archi.get_champions_f()[best_isl_idx]
print("Best relaxed solution, x: {}".format(x_best))
print("Best relaxed solution, f: {}".format(f_best))

Best relaxed solution, x: [761.24373095  46.75958218]
Best relaxed solution, f: [8.98238905]


In [ ]:
# Trying nevergrad again
import nevergrad as ng
from concurrent import futures

In [ ]:
instrum = ng.p.Instrumentation(
    ng.p.Array(shape=(2,)).set_bounds(lower=0, upper=100),
)

In [ ]:
# optimizer = ng.optimizers.CMA(parametrization=instrum, budget=1000, num_workers=12)

# with futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:
#     recommendation = optimizer.minimize(bla.fitness, executor=executor, batch_mode=False)

# recommendation

In [ ]:
from scipy.optimize import differential_evolution

In [ ]:
result = differential_evolution(bla.fitness, [(0,100),(0,100)], workers=12, popsize=15, maxiter=10)

In [ ]:
result.x

In [ ]:
# Baseline data
# opt_sample = [15, 2*np.sqrt(15)]

# Control optimisation
# opt_sample = [40.37586389, 10.79584889]
# opt_sample = [23, 21]
opt_sample = [40.70093065, 11.82625924]

# Motor optimisation
# opt_sample = [1.23963359e-01, 8.21454920e-03, 4.42151610e+00, 2.42814979e+01, 4.85943104e+00, 5.10276159e+03, 6.10012926e+00, 8.68464895e+01, 1.71650947e+01, 1.02976934e+00]

# Codesign optimisation
# opt_sample = [6.17907121e-01, 1.48666641e-01, 8.79340683e-01, 1.72980544e+01,3.85429047e+00, 5.52194647e+03, 2.56607694e+00, 9.39921750e+01,2.14105758e+01, 2.45136778e+00, 4.00414517e-02, 1.32539364e-01]

# Codesign with heuristics
# opt_sample = [1.75671160e+03, 2.42603688e+01, 9.78082909e+01, 1.38293459e+01,1.29247957e+00, 7.29913449e-02, 9.46402732e-02]

results = compute_QoI(pre_val, dv_names, opt_sample, case, run_sim = True)
print(results)
K_w = np.diag([1,1,1,1,100,5,5,0])
# Quadratic objective meta function with weights
fitness_val = np.matmul(K_w, results).dot(results)
print(fitness_val)

In [ ]:
def gripper_forward_kinematics_example():
    urdf_path = './urdfs/TwoLinkRobot.urdf'
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    # Transform for the robot location
    X_R = RigidTransform(RotationMatrix.MakeYRotation(np.pi/2), np.array([-0.1, 0.5, 1.0]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    # Spawn table|
    table = Parser(plant, scene_graph).AddModelFromFile('./urdfs/table/extra_heavy_duty_table_modified.sdf','table')
    # Spawn spherical work piece
    sphere = Parser(plant, scene_graph).AddModelFromFile('./urdfs/helper/sphere.urdf','sphere')
    X_R = RigidTransform(RotationMatrix(RollPitchYaw([0, 0, 0])), np.array([0, 0, 0.736]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(sphere)[0]).body_frame(), X_R)

    plant.Finalize()
    
    end_frame = "eef"
    
    # Draw the frames
    for body_name in ["base_link", end_frame]:
        AddMultibodyTriad(plant.GetFrameByName(body_name, model), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

#     plant.SetPositions(plant.GetMyContextFromRoot(context),
#                   plant.GetModelInstanceByName("diva_robot"),
#                   q0)
    
    gripper = plant.GetBodyByName(end_frame)
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())
    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()

In [ ]:
import nevergrad as ng
from concurrent import futures

In [ ]:
twoLinkprob = Baseline_Problem(pre_val, dv_names, dv_bounds, case)

In [ ]:
instrum = ng.p.Instrumentation(
    ng.p.Array(shape=(2,)).set_bounds(lower=0, upper=50),
)
optimizer = ng.optimizers.PSO(parametrization=instrum, budget=200, num_workers=12)

In [ ]:
with futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:
    recommendation = optimizer.minimize(twoLinkprob.fitness, executor=executor, batch_mode=False)

In [ ]:
recommendation

In [ ]:
2*np.sqrt(37.65)